In [1]:
import os

In [2]:
%pwd

'/Users/macbookpro/Desktop/pixi_hr_project/pixi_hr/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/macbookpro/Desktop/pixi_hr_project/pixi_hr'

# Entity
defining the return types of our data ingestion class.

Returns four variables, root directory, source url, local data file path, and unzip data path directory

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    # Directory where data ingestion artifacts are stored
    root_dir: Path
    # URL from which data will be downloaded
    source_URL: str
    # Path to the local file where downloaded data will be saved
    local_data_file: Path
    # Directory where the zipped data will be extracted
    unzip_dir: Path


In [6]:
from src.pixi_hr.constants import *
from src.pixi_hr.utils.common import read_yaml, create_directories

Update configuration manager inside src config, i.e. configuration.py

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,   # Path to the configuration YAML file
            params_filepath = PARAMS_FILE_PATH,   # Path to the parameters YAML file
            schema_filepath = SCHEMA_FILE_PATH):  # Path to the schema YAML file
        
        # Load the configuration details from the YAML file

        print(type(config_filepath), config_filepath)
        self.config = read_yaml(config_filepath)
        # Load the parameters details from the YAML file
        self.params = read_yaml(params_filepath)
        # Load the schema details from the YAML file
        self.schema = read_yaml(schema_filepath)


        # Create directories as specified in the configuration (e.g., for storing artifacts)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # Extract data ingestion configuration from the main configuration
        config = self.config.data_ingestion

        # Create directories specified in the data ingestion configuration
        create_directories([config.root_dir])

        # Create an instance of the DataIngestionConfig dataclass using the extracted configuration
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,               # Directory for data ingestion artifacts
            source_URL=config.source_URL,           # URL from which data will be downloaded
            local_data_file=config.local_data_file, # Local path where downloaded data will be saved
            unzip_dir=config.unzip_dir              # Directory where the zipped data will be extracted
        )

        return data_ingestion_config


We define the data ingestion configuration

Upate Components

In [8]:
import os
import urllib.request as request
import zipfile
from pixi_hr import logger
from pixi_hr.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize the DataIngestion class with the provided configuration.
        
        Args:
            config (DataIngestionConfig): Configuration for data ingestion.
        """
        self.config = config

    def download_file(self):
        """
        Downloads the data file from the specified source URL to a local path.
        If the file already exists locally, logs its size without re-downloading.
        """
        
        # Check if the data file already exists locally
        if not os.path.exists(self.config.local_data_file):
            # If not, download the file
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} with the following information: \n {headers}")
        else:
            # If file exists, log its size
            file_size = get_size(Path(self.config.local_data_file))
            logger.info(f"File already exists of size: {file_size}")

    
    def extract_zip_file(self) -> None:
        """
        Extracts the contents of the downloaded ZIP file to a specified directory.
        
        Returns:
            None
        """
        
        # Ensure the directory for extraction exists or create it
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        # Extract the contents of the ZIP file
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



Update Pipeline

In [10]:
try:
    # Initialize configuration manager to fetch relevant configurations
    config = ConfigurationManager()
    
    # Fetch the data ingestion-specific configuration
    data_ingestion_config = config.get_data_ingestion_config()
    
    # Initialize the data ingestion process with the fetched configuration
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Download the data file from the specified source
    data_ingestion.download_file()
    
    # Extract the downloaded ZIP file to the specified directory
    data_ingestion.extract_zip_file()

except Exception as e:
    # If any error occurs during the process, raise the exception to handle or log it appropriately
    raise e


<class 'pathlib.PosixPath'> config/config.yaml
[2023-08-13 16:24:41,803: 41: pixi_hr_project_logger: INFO: common:  yaml file: config/config.yaml loaded successfully]
[2023-08-13 16:24:41,804: 41: pixi_hr_project_logger: INFO: common:  yaml file: params.yaml loaded successfully]
[2023-08-13 16:24:41,805: 41: pixi_hr_project_logger: INFO: common:  yaml file: schema.yaml loaded successfully]
[2023-08-13 16:24:41,825: 64: pixi_hr_project_logger: INFO: common:  Created directory at: artifacts]
[2023-08-13 16:24:41,826: 64: pixi_hr_project_logger: INFO: common:  Created directory at: artifacts/data_ingestion]
[2023-08-13 16:24:43,405: 24: pixi_hr_project_logger: INFO: 1789850675:  Downloaded artifacts/data_ingestion/jobs_data_simply_hired.csv.zip with the following information: 
 Connection: close
Content-Length: 412936
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "213d138fc4fa6abfe638cff6cddad